<a href="https://colab.research.google.com/github/mphizi/SmartLandMaps/blob/main/SmartLandMaps_Grass___Malumbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install modules

Execute this cell first, then restart runtime. After restart, contuniue with the next cell.

*   Start installation
*   Restart runtime when finished






In [1]:
# Installing tesseract on the virtual machine
!sudo add-apt-repository ppa:alex-p/tesseract-ocr-devel -y

#install grass
!apt-get install grass -y;

!apt install libspatialindex-dev
!pip install rtree
!pip install geopandas

! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install pytesseract

import pytesseract
print(pytesseract.get_tesseract_version()) # should be above 5.2.0. 


Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:4 http://ppa.launchpad.net/alex-p/tesseract-ocr-devel/ubuntu focal InRelease [17.6 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,060 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,587 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,025 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:13 http://ppa.launchpad.net/de


Mount the drive folder on the notebook

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

# Set variables and paths

*   Attach the "scripts" folder to access the python scripts
*   Set Spatial Reference System with EPSG code
*   Specify input and output image and Json paths



In [ ]:
# VARIABLES

raster_name = 'ChadNorthern.tif'
epsg = 'EPSG:32633' # 'EPSG:32633' # Benin: 32631, Rwanda: 32735, Chad: 32633
path_scripts = '/content/gdrive/MyDrive/SmartLandMaps - AllDocs/Software/Systematic Testing/scripts/' # Path of the scripts folder
image_dir = '/content/gdrive/MyDrive/SmartLandMaps - AllDocs/Software/Systematic Testing/data/geotiff/Chad/' # Path of the folder conatining images
json_output_dir = '/content/gdrive/MyDrive/SmartLandMaps - AllDocs/Software/Systematic Testing/data/' # Path of folder for GeoJSON results
boundary_output_dir = '/content/gdrive/MyDrive/SmartLandMaps - AllDocs/Software/Systematic Testing/data/Chad boundaries/' # Path of folder for extracted boundary image
administrative_data_output_dir = '/content/gdrive/MyDrive/SmartLandMaps - AllDocs/Software/ChadPilot/data/attributes/' # Path of folder for the administrative data
logs_dir = '/content/gdrive/MyDrive/SmartLandMaps - AllDocs/Software/ChadPilot/data/logs/'
sticker_json = json_output_dir + raster_name[:-4]+'_num.json'

# From here onwards, parameters to retrieve the administrative data from odk
attributes_filename = 'chad_attributes.json'
credentials = {'email': 'auriol.degbelo@uni-osnabrueck.de', 'password': '*UsEr*2021*'}
url_post = "https://smartlmaps.mooo.com/v1/sessions"
#for benin url_get = "https://smartlmaps.mooo.com/v1/projects/4/forms/build_Untitled-Form_1643660577/submissions.csv" 
url_get = "https://smartlmaps.mooo.com/v1/projects/7/forms/1234567/submissions.csv" # url of the form to retrieve data from
saving_path = administrative_data_output_dir + attributes_filename


import sys
sys.path.append(path_scripts) 
input_raster = image_dir + raster_name




## **Boundary Extraction**

Load the input raster file, extract the boundaries and store the 
extracted boundaries as a tif file



In [ ]:
patch_shape = (640, 640, 3) # the size of the patches, the first two parameters should be multiples of 32, the third is always 3
current_threshold = 45 # between 0 and 255. Threshold for the greyscale image, lower value means less noise but less pixels in boundaries
rgba = False # change to True if the input image is a rgba image
tfw = True # change to False if you don't want to generate a tfw file along with the tif for the boundaries
plot = False # change to True if you want the patches visualized

!pip install patchify
# Don't need to change these
boundaries_output = boundary_output_dir + raster_name[:-4] + "_bound_"+str(current_threshold)+".tif"
parcel_poly = json_output_dir + raster_name[:-4] + '_poly_'+str(current_threshold)+'.json'

#boundary_extractor_mean_2023_02
from boundary_extractor_blur_2023_02 import extract_boundaries 

#extract_boundaries(input_raster, boundaries_output, patch_shape, current_threshold, rgba, tfw, plot) 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ModuleNotFoundError: ignored

## **Vectorization, Postprocessing and Geojson Export**

Generates polygons based on the boundary raster, performs generalization and exports them as a geojson file.

In [ ]:
min_parcel_area = 50 # Minimum area of pacels in square meters (if set to 0 all polygons are extracted) 
runs_num = 1 # Number of runs. ### If you want to run this part again change it or else GRASS won't work.###
douglas_thresh = 0.000005 # Threshold for the Doulgas-Peucker algorithm. 0.000005 gives good result for around 500m^2 parcels. 
delraw = False # If set to True, the raw polygons GeoJSON is deleted, if False, it is kept.

!pip install pyproj
!pip install geojson 
!pip install grass-session

from vectorizer_grass_2023_02 import generate_polygons

generate_polygons (input_raster, boundaries_output, epsg,  parcel_poly, min_parcel_area, runs_num, douglas_thresh, delraw)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ModuleNotFoundError: ignored

## **Sticker Extraction and Geojson Export**


Loads the input raster file, extracts the stickers and their locations, and returns the detected numbers with coordinates as a geojson file



In [ ]:
#Thresholds for colour segmentation 0-255 for each channel (B G R)
bmin=0
bmax=120
gmin=140
gmax=255
rmin=170
rmax=255
# Set these to filter sticker pixels from the background. Leave some margin so that stickers don't erodate 
# and numbers can be seen clearly, but try to set them to remove the most noise possible.

!pip install pyproj
!pip install geojson
!pip install imutils

from OCR_colab_2023_02 import detect_stickers
detect_stickers(input_raster, sticker_json, bmin, bmax, gmin, gmax, rmin, rmax, epsg)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading image...
Done. Preprocessing...
Done. Detecting contours...
Done. Extraction and OCR...
Done. Number of stickers extracted: 59
Writing results...
Finished


## **Analyse relations between extracted and ODK data**



In [ ]:
#Analyses the results and gives feedback on the amount of parcels or stickers that need editing. 
#Also compares sticker ID-s with an ODK Form and checks for inconsistency. Output is written into a log file.

from analyser_2023_02 import retrieve_odk_administrative_data, get_token, analyse_relations
#!pip install geopandas # in case you're not running every block from the start

odk_data = retrieve_odk_administrative_data (url_get, get_token(url_post, credentials), saving_path)
analyse_relations(raster_name, sticker_json, parcel_poly, odk_data, logs_dir)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.8 MB/s 
     |████████████████████████████████| 16.6 MB 104 kB/s 
Response Code:  200
Filename: Mi_northern_HighRes_clip.tif
Number of parcels detected: 57
Number of stickers detected: 55
Number of records in odk: 83
Number of parcels containing exactly 1 sticker: 55
No duplicate records in detected numbers
Number of parcels that need to be checked: 2
Number of stickers that need to be checked: 0
Polygon with parcelID 0 contains 0 stickers.
Polygon with parcelID 45 contains 0 stickers.
Number of records detected (55) does not match with number of records in odk (83)
ID-s match odk data, no check needed


## **Join parcels with ODK data**

In [ ]:
# Join ODK data with polygons by sticker ID
# Before running check if data needs to be edited

# paths to change:
# You can delete all the intermediate results as the last step. The finaloutput is what is important. 
stickers_json="output/numtest.geojson"
parcels_json="output/tab1gr_80_gen1cl.geojson"
polyidout='output/test_polyid.json' 
finaloutput="output/"+'test_parcels_with_attributes.json'

from geopandas import gpd 

points = gpd.GeoDataFrame.from_file(stickers_json) # Stickers JSON  
polys = gpd.GeoDataFrame.from_file(parcels_json) # Parcels JSON
polyid = gpd.sjoin(polys, points, how='left') 
polyid.to_file(polyidout) # Polygons with ID-s output filename

print("Sticker ID successfully merged with polygons")

    # Join ODK data with polygons by sticker ID
# Before running check if data needs to be edited
import pandas as pd

odk_dataframe = pd.DataFrame.from_dict(odk_data)
odk_poly_merged = polyid.merge(odk_dataframe, left_on='Sticker_number', right_on='SpatialUnit-ID')
odk_poly_merged.to_file(finaloutput) # Parcels with all attributes from odk


Response Code:  200


## **View ODK data**

In [ ]:
import random 

# Extract information from the ODK data
from analyser_2023_02 import retrieve_odk_administrative_data, get_token

odk_data = retrieve_odk_administrative_data (url_get, get_token(url_post, credentials), saving_path)

print("Number of attribute data collected: ", len(odk_data))
print("Printing now an example of attribute data...")
print(random.choice(odk_data))

Response Code:  200
Number of attribute data collected:  83
Printing now an example of attribute data...
OrderedDict([('SubmissionDate', '2022-10-11T16:49:17.832Z'), ('FirstName', 'Tchigo'), ('Surname', 'Mahamat  Abakar '), ('ID_number', '28'), ('Gender', 'Homme'), ('DateNaissance', '1971-01-01'), ('Nationalite', 'Tchadien'), ('Representante', 'Oui'), ('PartyType', 'Personne physique'), ('EtatCivil', 'Marié'), ('LandRight', "Droit d'occupation"), ('RightSource', 'Achat'), ('ShareFraction', '100'), ('Statut', "N'existe pas dans la base de donnees"), ('SpatialID', '28'), ('DataCollector', 'Fakadi solange '), ('Methodologie', 'Participatory Mapping'), ('LandUse', 'Residentiel'), ('Conflit', 'Non'), ('meta-instanceID', 'uuid:c95dcf1f-4f0d-493c-812d-26e4e644f196'), ('KEY', 'uuid:c95dcf1f-4f0d-493c-812d-26e4e644f196'), ('SubmitterID', '57'), ('SubmitterName', 'Kaspar'), ('AttachmentsPresent', '0'), ('AttachmentsExpected', '0'), ('Status', ''), ('ReviewState', ''), ('DeviceID', 'collect:a1SYv